In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import matplotlib.image as Image
import keras
%matplotlib inline
import bcolz
from scipy.misc import imresize
if utils is None:
    import utils
else:    
    reload(utils)
    print 'utils has been reloaded'
from datetime import datetime, timedelta

In [ ]:
# EMBEDDING CLASS TO BE USED
class Embedding:
    def __init__(self, topology='vgg16'):
        self.topology = topology
        self.network = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet',
                                                      input_tensor=None,input_shape=None,
                                                      pooling=None, classes=1000)
        self.network.layers.pop() # Get rid of the classification layer
        self.network.outputs = [self.network.layers[-1].output]
        self.network.layers[-1].outbound_nodes = []
        for layer in self.network.layers: layer.trainable=False
    def get_vector(self, batch):
        return self.network.predict(batch)

In [ ]:
a = np.arange(10)
b = bcolz.carray(a)
c = bcolz.carray(a, rootdir='data')
c.flush()

In [ ]:
API_KEY = 'J9rmStnqj0ZhQFObVxh81O84hb7UQZjY'
N_FILES = 100
SEARCH_TERM = 'cat'

In [ ]:
api_path = "http://api.giphy.com/v1/gifs/search?q={}&api_key={}&limit={}".format(SEARCH_TERM, API_KEY, N_FILES)
data = json.loads(urllib.urlopen(api_path).read())

In [ ]:
utils.save_files(data,SEARCH_TERM)

In [ ]:
import os
for gif_file in os.listdir('gifs'):
    print 'gifs/'+gif_file

In [ ]:
gif_file = 'cat_93.gif'
gif_file_path = 'gifs/'+gif_file
frames = utils.processImage(gif_file_path, reshape_to_vgg=True)
np.shape(frames)

pyplot.imshow(frames[0])

In [ ]:
import h5py as h5py

In [ ]:
embeddings = Embedding()

In [ ]:
start_time = datetime.now()
embedding_matrix = embeddings.get_vector(data)
round(timedelta.total_seconds(datetime.now()-start_time)/60,2)

In [ ]:
start_time = datetime.now()
embedding_matrix = embeddings.get_vector(frames)
round(timedelta.total_seconds(datetime.now()-start_time)/60,2)

In [ ]:
embedding_matrix.shape
# in numpy slicing, last element is exclusive
# ex x[0:2,:] = x[[0,1],:]
max_index = embedding_matrix.shape[0]
seq_length = 10
data_dict = {}
data_arr = []
target_arr = []
for i in range(max_index-seq_length):
    start_ix = i
    end_ix = i+seq_length
    target_ix = end_ix
    data_arr.append(embedding_matrix[start_ix:end_ix,:])
    target_arr.append(embedding_matrix[[target_ix],:])

In [ ]:
np.array(data_arr).shape,np.array(target_arr).shape

### Preparing the data for the LSTM

In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [34]:
def toy_input_fn():
    x = np.array([np.random.randint(high=255, low=0, size=(10,255))
                 for _ in range(5)])
    y = np.array([np.random.randint(high=255, low=0, size=(255))
                  for _ in range(5)])
    
    return x, y                   

In [35]:
x, y = toy_input_fn()
x.shape, y.shape

((5, 10, 255), (5, 255))

In [75]:
INPUT_EMBEDDING_SIZE = 255
OUTPUT_EMBEDDING_SIZE = 255
IS_TRAINING = True
N_HIDDEN = 512
N_UNITS = 2
N_STEPS = 10
BATCH_SIZE = 5

In [76]:
def build_model(output_embedding_size=OUTPUT_EMBEDDING_SIZE,
                input_embedding_size=INPUT_EMBEDDING_SIZE,
                is_training=IS_TRAINING,
                n_hidden=N_HIDDEN,
                n_units=N_UNITS,
                n_steps=N_STEPS,
                batch_size=BATCH_SIZE):
    """
    Builds the model for the LSTM model
    :param vocab_size: Size of the vocab from the input function
    :param is_training: Boolean, true when training
    :param n_hidden: N hidden units
    :param n_units: N LSTM units
    :param embedding_size: Size of the embedding dimensions
    :param n_steps: N timesteps we train over
    :return:
           predictions: Softmax prediction over words
           loss: Loss from current batch
           x: Placeholder for inputs
           y: Placeholder for labels
    """
  
    with tf.name_scope("placeholders"):
        x = tf.placeholder(dtype=tf.int32, shape=[None, n_steps, input_embedding_size])
        y = tf.placeholder(dtype=tf.int32, shape=[None, output_embedding_size])
        seq_length = tf.placeholder(tf.int32, [None])

    # Creates the actual model using embeddings as input
    with tf.name_scope("model"):
        lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_hidden)
                      for _ in range(n_units)]
       
        multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
        inputs = tf.cast(x, tf.float32)
        outputs, states = tf.nn.dynamic_rnn(multi_cell, inputs, dtype=tf.float32)
        top_layer_h_state = states[-1][1]

        hidden1 = tf.layers.dense(top_layer_h_state, units=n_hidden, activation=tf.nn.relu)
        dropout_1 = tf.layers.dropout(hidden1, rate=0.1, training=is_training)
        predictions = tf.layers.dense(dropout_1, units=output_embedding_size, activation=None)
        mse = tf.losses.mean_squared_error(
            labels=y,
            predictions=predictions)
        loss = tf.reduce_mean(mse)

        return predictions, loss, x, y

In [80]:
g = tf.Graph()
with g.as_default():
    sess = tf.InteractiveSession()
    x_batch, y_batch = toy_input_fn()
    predictions, loss, x, y = build_model()
    sess.run(tf.global_variables_initializer())
    predictions_out, loss_out = sess.run([predictions, loss], feed_dict={x: x_batch, y: y_batch})
    print(predictions_out)

[[-0.18163717 -0.09708181 -0.04113833 ...,  0.1468585  -0.16404206
  -0.2442089 ]
 [-0.19735549 -0.00987558 -0.11963176 ...,  0.30203173 -0.04542725
  -0.22298522]
 [-0.20527643  0.06213007 -0.05125128 ...,  0.30264187  0.08494422
  -0.12003544]
 [-0.19180053 -0.03962021  0.02120179 ...,  0.07759659  0.04699408
  -0.2415058 ]
 [-0.27015352  0.03498184 -0.03082781 ...,  0.15595968  0.10825367
  -0.22291814]]
